## **BBM 444 - Programming Assignment 2: HDR Imaging and Tonemapping**

* You can add as many cells as you want in-between each question, define any function and variable as needed. However, your code needs to be well-commented.  
* Please add Markdown cells to answer the (non-coding) questions in the homework text.
* Please be careful about the order of runs of cells. Doing the homework, it is likely that you will be running the cells in different orders, however, they will be evaluated in the order they appear. Hence, please try running the cells in this order before submission to make sure they work.    
* Please refer to the homework text for any implementation detail. Though you are somewhat expected to abide by the comments in the below cells, they are mainly just provided for guidance and by no means are they complete and/or descriptive enough. Accordingly, as long as you are not completely off this structure and your work pattern is understandable and traceable, it is fine. For instance, you do not have to implement a particular function within a cell just because the comment directs you to do so. 
* **Required packages and functions:** numpy, skimage, matplotlib, and cv2 (OpenCV, to read and write HDR files), and you should use the functions provided in the ./ src/cp_assng2.py file of the assignment ZIP archive.
* Working with the provided and captured exposure stacks, you will notice that your algorithms will be using a lot of memory. That's why you should be careful about how many of these images you create in your Python code, as otherwise you run the risk of filling up your memory. Further, you need to make sure you use vectorized code that processes all of its pixels in parallel, as trying to process all 25 million pixels one-by-one with a double for loop will take ages.
* To be able to display HDR images, multiply your image with an appropriate scaling factor of your own (smaller than 1 if the image is very bright, larger than 1 otherwise), apply gamma encoding, and then use the clip and imshow functions as in Programming Assignment 1. You will likely need to experiment with a few different values for the scaling factor you apply, until you find the one that correctly exposes your image. Otherwise, it may appear very bright or very dark.
* There will be a lot of reusing the same functionality, hence implementing the algorithms you are asked with functions (and making appropriate calls when necessary, rather than just code blocks ) is likely to be beneficial.
* You can use the function readHDR in the code we provide to open
and load the .HDR in Python, as otherwise to open HDR images, you'll need a dedicated viewer.
* **This document is also your report. Show your work. Please articulate your arguments well and support them using relevant figures and images.**



### *Insert personal info here*

### 1. HDR Imaging (60 points)

You are provided with both RAW (with .NEF extensions) and rendered (with .JPG extensions) images in the data folder, both of which are to be used to create HDR images.

#### 1.1. Develop RAW images (5 points)

To convert RAW .NEF images into linear 16-bit .TIFF images, use dcraw and specify the camera's profile for white balancing, high-quality interpolation for demosaicing, and sRGB as the output color space. The correct set of flags for this conversion can be found in dcraw's documentation. Report them in the below cell.

##### ***REPORT dcraw flags here. Double tap to write in this cell.***


"-v": "Verbose mode - prints detailed processing information",

"-w": "Use camera white balance (as seen in output 'multipliers 1.058594 1.000000 2.500000 1.000000')",

"-4": "Write 16-bit linear output (for linear TIFF requirement)",

"-t 0": "High-quality interpolation for demosaicing (uses AHD - Adaptive 
Homogeneity-Directed interpolation)",

"-T": "Write TIFF output instead of PPM",

"-j": "Use camera color profile and convert to sRGB colorspace (as seen in 'Converting to sRGB colorspace')"


## OUTPUT

Loading NIKON D2X image from data/Lab Booth/_MDF0019.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0019.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0020.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0020.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0021.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0021.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0022.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0022.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0023.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0023.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0024.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0024.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0025.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0025.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0026.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0026.tiff ...


Loading NIKON D2X image from data/Lab Booth/_MDF0027.NEF ...
Scaling with darkness 0, saturation 4095, and
multipliers 1.058594 1.000000 2.500000 1.000000
AHD interpolation...
Converting to sRGB colorspace...
Writing data to data/Lab Booth/_MDF0027.tiff ...


#### 1. 2. Weighting Schemes

You are expected to implement 4 different weighting schemes, namely, uniform, tent, Gaussian, and photon. (All hold the assumption that the intensity values z $\in$ [0,1]).

\\begin{eqnarray}
w_{\text{uniform}} & = & \left\{
        \begin{array}{ll}
            1, & \;\, \text{if}\;\, Z_{\text{min}}\leq z \leq Z_{\text{max}}, \\
            0, & \;\, \text{otherwise}
        \end{array},
    \right.\\\nonumber
w_{\text{tent}} & = & \left\{
        \begin{array}{ll}
            \text{min}(z,1-z), & \;\, \text{if}\;\, Z_{\text{min}}\leq z \leq Z_{\text{max}}, \\
            0, & \;\, \text{otherwise}
        \end{array},
    \right. \\\nonumber
w_{\text{Gaussian}} & = & \left\{
        \begin{array}{ll}
            \text{exp}\left(-4\frac{(z-0.5)^2}{0.5^2}\right), & \;\, \text{if}\;\, Z_{\text{min}}\leq z \leq Z_{\text{max}}, \\
            0, & \;\, \text{otherwise}
        \end{array},
    \right. \\\nonumber
w_{\text{photon}} & = & \left\{
        \begin{array}{ll}
            t^k, & \;\, \text{if}\;\, Z_{\text{min}}\leq z \leq Z_{\text{max}}, \\
            0, & \;\, \text{otherwise}
        \end{array},
    \right.
\end{eqnarray} 

Though the recommended values for $Z_{\text{max}}$ and $Z_{\text{min}}$ are 0.05 and 0.95 respectively, you can experiment with different clipping values. 


 

In [1]:
import os


In [2]:
os.chdir("assignments/assignment2-cmp")

In [3]:
## Implement (possibly) functions or a class for these weighting schemes
from weighting_schemes import WeightingSchemes




ModuleNotFoundError: No module named 'weighting_schemes'

#### 1.3. Linearize rendered images (25 points)

Rendered images, being non-linear, need to be linearized before merging into HDR images. For this, we'll use the method by Debevec and Malik [1]. Please refer to the homework text and the paper for any implementation detail.

$I^k_{ij}: $ intensity of pixel (i, j) of the $k^{th}$ image and $I^k_{ij} = f(t^kL_{ij}) $, where $I^k_{ij} \in \{ 0, ..., 255 \}$ and $t^k$ the exposure time of image k. f is the non-linearity introduced by the camera's ISP. That is, calculating its inverse, $f^{-1}$, one can get the linear response.

Instead of $f^{-1}$, recover the function $g =\text{log}(f^{-1})$ that maps the pixel values $I^k_{ij}$ to $g(I^k_{ij})=\text{log}(L_{ij})+\text{log}(t^k)$. **Notice domain of g is discrete** an takes on values in the range $\{ 0, ..., 255 \}$. Hence, the second derivative of g can be approximated via a Laplacian filter.

Assuming static scene upon capturing, $L_{ij}$ is constant across all LDR images. That is, to recover g, the least-squares optimization problem in the hw text needs to be solved. 

*Hint: Solve problem by expressing it in matrix form: $||Av -b ||^2$, where A is a matrix and $\mathbf{v} = [g; \text{log} (L_{ij} )]$ are the unknowns. Use one of NumPy’s solvers to recover the unknowns. (See numpy function numpy.linalg.lstsq)* 

Though, Debevec and Malik [1] recovers a g for each color channel, recovering a single g for all channels suffices for this homework.

Plot the function g you recovered and use it to convert non-linear images into linear.

**Solving the linear equation, unless downsampled, processing the whole image will cause memory error, ie, you will run out of memory. That's why you need to downsample the image I with I[::N, ::N], for some N . We recommend using N = 200. In general it is advisable to use downscaled images during the debugging process of your code to speed up the development process. After ensuring the correctness of the code, the final results can be obtained by running it on the full-resolution image.**


In [4]:
## Implement a solver (perhaps a function) for the optimization problem and recovering g
from response_calibration import ResponseCalibration

ModuleNotFoundError: No module named 'response_calibration'

#### 1.4. Merge exposure stack into HDR image (30 points)
Given a set of k LDR linear images corresponding to different exposures $t^k$, we can merge them into an HDR image either in the linear or in the logarithmic domain, where the motivation for the former is physical accuracy, whereas, that for the latter is human visual perception.

Please see the homework text for both algorithms. 

**Merging multiple LDR images, some pixels may not have well-exposed values, which makes the sum of weights in the equations' denominators zero. For over-exposed pixels, assign the maximum valid pixel value, and for under-exposed pixels, assign the minimum valid pixel value.**

In [5]:
from hdr_merging import HDRMerger


ModuleNotFoundError: No module named 'hdr_merging'

#### 1.4. Experiment

You have 2 sets of images (RAW and rendered), 2 merging schemes (linear and logarithmic), and 4 weighting schemes (uniform, tent, Gaussian, and photon-noise optimal), which, in total, makes 16 different HDR images. Additionally, you will need to tune the regularizer hyperparameter $\lambda$.  

Select one out of the sixteen HDR images you created. You can select, for
example, the one that you find the most aesthetically pleasing. Make sure to comment on why you selected the image. Note that, as you have not yet tonemapped your HDR images, if you display them directly they will not look very nice; see “Hints and Information”.

In [6]:
from hdr_experiments import *

ModuleNotFoundError: No module named 'hdr_experiments'

In [7]:
os.listdir()

['_MDF0021.NEF',
 '_MDF0019.NEF',
 '_MDF0023.NEF',
 '_MDF0024.NEF',
 '_MDF0027.NEF',
 '_MDF0021.tiff',
 '_MDF0027.tiff',
 '_MDF0020.NEF',
 '_MDF0026.NEF',
 '_MDF0019.tiff',
 '_MDF0026.tiff',
 '_MDF0023.tiff',
 '_MDF0020.tiff',
 '_MDF0021.jpg',
 '_MDF0027.jpg',
 '_MDF0022.tiff',
 '_MDF0024.tiff',
 '_MDF0023.jpg',
 '_MDF0022.jpg',
 '_MDF0020.jpg',
 '_MDF0025.tiff',
 '_MDF0026.jpg',
 '_MDF0019.jpg',
 '_MDF0024.jpg',
 '_MDF0025.NEF',
 '_MDF0022.NEF',
 '_MDF0025.jpg']

In [8]:
try:
    # Import required modules
    import os
    import traceback
    from hdr_experiments import run_experiments, visualize_results
    
    print("Starting experiments...")
    
    # Run the experiments (path handling is already done in hdr_experiments.py)
    results = run_experiments()
    
    # Visualize the results
    print("\nGenerating visualizations...")
    visualize_results(results)
    
    print("\nExperiments completed successfully!")

except Exception as e:
    print(f"\nAn error occurred: {str(e)}")
    print("\nDetailed error information:")
    print(traceback.format_exc())


An error occurred: No module named 'hdr_experiments'

Detailed error information:
Traceback (most recent call last):
  File "/tmp/ipykernel_1509/1639103817.py", line 5, in <module>
    from hdr_experiments import run_experiments, visualize_results
ModuleNotFoundError: No module named 'hdr_experiments'



In [8]:
## Plot g

#### *Choose one of the HDR images you have created, commenting on the reason for your choice.*

*Store the resulting HDR images as \texttt{.HDR} files, which is an open source high dynamic range file format. (See the provided function **`writeHDR`**  ./src/cp\_assgn2.py)*

## 2. Color correction and white balancing (20 points)

For this part, you are expected to use the **read_colorchecker_gm()** function provided in **./src/cp_assgn2.py**., which **returns a 4x6 matrix with sRGB linear values of the Greatg-Macbeth color checker.** 



1. For each color checker patch, crop a square that is fully contained within the patch. (See mat-plotlib function matplotlib.pyplot.ginput for interactively recording image coordinates). Make sure to store the coordinates of these cropped squares, so that you can re-use them. Use the resulting 24 crops to compute average RGB coordinates for each of the color checker’s 24 patches
2. Convert these computed RGB coordinates into homogeneous 4 × 1 coordinates, by appending a 1 as their fourth coordinate.
3. Solve a least-squares problem to compute an affine transformation, mapping the measured to the ground-truth  homogeneous coordinates.
4. Apply the computed affine transform to your original RGB HDR image. Note that the
transformed image may have some negative values, which you should clip to 0.
5. Finally, apply an additional white balancing transform (i.e., multiply each channel with a scalar), so that the RGB coordinates of patch 4 are equal to each other. This is analogous to the manual white balancing in Programming Assignment 1, where now we use patch 4 as the white object in the scene.

Store the color corrected and white balanced HDR image in an .HDR file. You should now have two HDR images total: The one from Part 1 that has not been color-corrected, and the one you just created. Compare the color-corrected image with the original, and discuss which one you like the best.


In [9]:
# for each color patch: get 24 crops
def crop_color_patches(image, interactive=True, saved_coordinates=None):
    """
    Crop 24 patches from color checker, either interactively or using saved coordinates.
    
    Args:
        image (numpy.ndarray): Input HDR image
        interactive (bool): Whether to use interactive selection or saved coordinates
        saved_coordinates (list): List of (x, y, size) tuples for each patch if not interactive
        
    Returns:
        tuple: (patches, coordinates) where patches is a list of 24 cropped regions and
              coordinates is a list of (x, y, size) tuples
    """
    patches = []
    coordinates = []
    
    if interactive:
        # Display the image for interactive selection
        plt.figure(figsize=(12, 8))
        plt.imshow(np.clip(image, 0, 1))  # Clip for display purposes
        plt.title("Select the center of each color patch (24 points total)")
        plt.axis('on')
        
        # Get 24 points from user interaction
        points = plt.ginput(24, timeout=0)
        plt.close()
        
        # Define patch size (you may need to adjust this)
        patch_size = min(image.shape[0], image.shape[1]) // 20
        
        # Extract patches
        for x, y in points:
            x, y = int(x), int(y)
            half_size = patch_size // 2
            
            # Ensure the patch is within image boundaries
            x_start = max(0, x - half_size)
            y_start = max(0, y - half_size)
            x_end = min(image.shape[1], x + half_size)
            y_end = min(image.shape[0], y + half_size)
            
            patch = image[y_start:y_end, x_start:x_end]
            patches.append(patch)
            coordinates.append((x, y, patch_size))
    else:
        # Use saved coordinates
        if saved_coordinates is None or len(saved_coordinates) != 24:
            raise ValueError("Need exactly 24 saved coordinates")
        
        for x, y, size in saved_coordinates:
            half_size = size // 2
            x_start = max(0, x - half_size)
            y_start = max(0, y - half_size)
            x_end = min(image.shape[1], x + half_size)
            y_end = min(image.shape[0], y + half_size)
            
            patch = image[y_start:y_end, x_start:x_end]
            patches.append(patch)
            coordinates.append((x, y, size))
    
    return patches, coordinates

In [10]:
# compute average RGB coordinates for each of the color checker’s 24 patches and convert them to 4x1 coords by adding a 1
def compute_average_rgb(patches):
    """
    Compute average RGB values for each patch.
    
    Args:
        patches (list): List of 24 cropped patch regions
        
    Returns:
        numpy.ndarray: 24x3 array of average RGB values
    """
    avg_rgb = []
    
    for patch in patches:
        # Compute average RGB for each patch
        avg_color = np.mean(patch, axis=(0, 1))
        avg_rgb.append(avg_color)
    
    return np.array(avg_rgb)

In [11]:
# compute an affine transformation
def convert_to_homogeneous(rgb_values):
    """
    Convert RGB values to homogeneous coordinates by appending 1.
    
    Args:
        rgb_values (numpy.ndarray): Nx3 array of RGB values
        
    Returns:
        numpy.ndarray: Nx4 array of homogeneous coordinates
    """
    # Add a column of ones to make homogeneous coordinates
    ones = np.ones((rgb_values.shape[0], 1))
    return np.hstack((rgb_values, ones))


In [12]:
# Apply the computed affine transform to your original RGB HDR image
def compute_affine_transform(source_coords, target_coords):
    """
    Compute affine transformation matrix using least squares.
    
    Args:
        source_coords (numpy.ndarray): Nx4 homogeneous coordinates of measured values
        target_coords (numpy.ndarray): Nx3 ground truth RGB values
        
    Returns:
        numpy.ndarray: 4x3 affine transformation matrix
    """
    # Solve for each channel separately
    transformation = np.zeros((4, 3))
    
    for i in range(3):  # For R, G, B channels
        # Solve least squares: source_coords * X = target_coords[:, i]
        x, residuals, rank, s = np.linalg.lstsq(source_coords, target_coords[:, i], rcond=None)
        transformation[:, i] = x
    
    return transformation

def apply_affine_transform(image, transform):
    """
    Apply affine transformation to an HDR image.
    
    Args:
        image (numpy.ndarray): Input HDR image
        transform (numpy.ndarray): 4x3 affine transformation matrix
        
    Returns:
        numpy.ndarray: Color-corrected HDR image
    """
    # Reshape image to 2D array of pixels
    h, w, c = image.shape
    pixels = image.reshape(-1, c)
    
    # Convert to homogeneous coordinates
    homogeneous = np.hstack((pixels, np.ones((pixels.shape[0], 1))))
    
    # Apply transformation
    corrected = np.dot(homogeneous, transform)
    
    # Reshape back to image
    corrected_image = corrected.reshape(h, w, c)
    
    # Clip negative values
    corrected_image = np.maximum(corrected_image, 0)
    
    return corrected_image

In [13]:
# Apply white balancing
def apply_white_balance(image, patch4_avg):
    """
    Apply white balancing transform so that the RGB coordinates of patch 4 are equal.
    
    Args:
        image (numpy.ndarray): Input HDR image
        patch4_avg (numpy.ndarray): Average RGB value of patch 4 (white patch)
        
    Returns:
        numpy.ndarray: White-balanced HDR image
    """
    # Get the maximum value across channels of patch 4
    max_val = np.max(patch4_avg)
    
    # Compute scaling factors to make R=G=B for patch 4
    scaling = max_val / patch4_avg
    
    # Apply scaling to entire image
    balanced_image = np.zeros_like(image)
    for i in range(3):
        balanced_image[:, :, i] = image[:, :, i] * scaling[i]
    
    return balanced_image

In [14]:
# Store the color corrected HDR image
def save_hdr_image(image, filename):
    """
    Save an HDR image to an HDR file.
    
    Args:
        image (numpy.ndarray): HDR image to save
        filename (str): Output filename
    """
    # OpenCV expects BGR order for HDR files
    bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(filename, bgr_image)

In [15]:
def color_correct_hdr_image(hdr_image, color_checker_values, interactive=True, saved_coordinates=None):
    """
    Complete color correction and white balancing pipeline.
    
    Args:
        hdr_image (numpy.ndarray): Input HDR image
        color_checker_values (numpy.ndarray): 4x6x3 ground truth values from color checker
        interactive (bool): Whether to select patches interactively
        saved_coordinates (list): Previously saved coordinates if not interactive
        
    Returns:
        tuple: (color_corrected_image, white_balanced_image, coordinates)
    """
    # Reshape color checker values to 24x3
    ground_truth = color_checker_values.reshape(24, 3)
    
    # 1. Crop color patches
    patches, coordinates = crop_color_patches(hdr_image, interactive, saved_coordinates)
    
    # 2. Compute average RGB for each patch
    avg_rgb = compute_average_rgb(patches)
    
    # 3. Convert to homogeneous coordinates
    homogeneous_coords = convert_to_homogeneous(avg_rgb)
    
    # 4. Compute affine transformation
    transform = compute_affine_transform(homogeneous_coords, ground_truth)
    
    # 5. Apply affine transformation
    color_corrected = apply_affine_transform(hdr_image, transform)
    
    # 6. Apply white balancing (patch 4 is the 4th patch in the first row, index 3)
    patch4_avg = avg_rgb[3]
    white_balanced = apply_white_balance(color_corrected, patch4_avg)
    
    return color_corrected, white_balanced, coordinates

##### *Compare the color-corrected and the original image here, discussing which one you like better.*

In [16]:

def visualize_color_correction(original, color_corrected, white_balanced):
    """
    Visualize original, color-corrected, and white-balanced images for comparison.
    
    Args:
        original (numpy.ndarray): Original HDR image
        color_corrected (numpy.ndarray): Color-corrected HDR image
        white_balanced (numpy.ndarray): White-balanced HDR image
    """
    # Display function for HDR images
    def display_hdr(img):
        # Simple tone mapping for display
        gamma = 2.2
        img_display = np.clip(img, 0, None)
        img_display = (img_display / (np.max(img_display) + 1e-6))**(1/gamma)
        return np.clip(img_display, 0, 1)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(display_hdr(original))
    axes[0].set_title('Original HDR')
    axes[0].axis('off')
    
    axes[1].imshow(display_hdr(color_corrected))
    axes[1].set_title('Color Corrected')
    axes[1].axis('off')
    
    axes[2].imshow(display_hdr(white_balanced))
    axes[2].set_title('White Balanced')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show() 

### 3. Photographic tonemapping (20 points)

You need to tonemap the HDR image you like better at the end of the last part for displaying purposes.
You will implement the tonemapping operator proposed by Reinhard et al. [2].
For implementation details, refer to the homework text and the paper.

*You may get better results by using the same scalars for all three channels. You can do this by using pixels from all three channels in the equations.*

$I_{white} = B. max_{i,j}(I_{ij,HDR})$,

$I_{i,j,HDR} = \frac{K}{I_{m,HDR}}$

$I_{m,HDR} = exp(1/N∑_{i,j}log(I_{ij,HDR} + ϵ))$ **Equation (10)**

The parameter K is the key, and determines how bright or dark the resulting tonemapped rendition is. The parameter B is the burn, and can be used to  suppress the contrast of the result. Finally, N is the number of pixels, and ε is a small constant to avoid the singularity of the logarithm function at 0. 

**Even with tonemapping, your images may appear too dark. After tonemapping, you need to apply gamma encoding for images to be displayed correctly.**


### 4. (Bonus)  Create and tonemap your own HDR photos (50 points)

* Apply your implementation on a photograph you have taken. For this, you need to choose a scene with high dynamic range. See the hints section in the homework text for the camera settings.
* Once you select the scene, capture exposure stacks in RAW and JPEG formats. We suggest using exposures that are equally spaced in the logarithmic domain. For example, start with some very low base exposure, and then use exposures that are 2× the base, 4×, 8×, and so on.
* Use the exposure stacks you captured to create two HDR images, one from the RAW and one from the JPEG images. Store these images in .HDR format. You do not need color calibration. 
*Then, process these images using the tonemapping algorithms you implemented in Part 3 (photographic, in RGB or luminance-only). Experiment with different parameters, show a few representative tonemaps, discuss your results, and determine which result you like the most.

In [17]:
## Load your images 


def load_exposure_stack(directory, file_format):
    """
    Load an exposure stack of images from a directory.
    
    Args:
        directory (str): Directory containing exposure stack images
        file_format (str): File format ('RAW', 'JPEG', 'NEF', 'JPG', etc.)
        
    Returns:
        tuple: (images, exposure_times) where images is a list of image arrays
               and exposure_times is a list of corresponding exposure times
    """
    # Get image file paths
    if file_format.upper() == 'RAW' or file_format.upper() == 'NEF':
        pattern = os.path.join(directory, f'*.NEF')
    else:  # JPEG/JPG
        pattern = os.path.join(directory, f'*.jpg')
        if not glob.glob(pattern):
            pattern = os.path.join(directory, f'*.jpeg')
    
    # Get sorted list of files
    file_paths = sorted(glob.glob(pattern))
    
    if len(file_paths) == 0:
        raise ValueError(f"No {file_format} files found in {directory}")
    
    print(f"Loading {len(file_paths)} {file_format} images...")
    
    # Load images
    images = []
    for file_path in file_paths:
        print(f"  Loading {os.path.basename(file_path)}")
        if file_format.upper() == 'RAW' or file_format.upper() == 'NEF':
            # For RAW files, use cv2.imread with ANYDEPTH flag
            img = cv2.imread(file_path, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_ANYCOLOR)
        else:
            # For JPEG/JPG files
            img = cv2.imread(file_path)
        
        # Convert BGR to RGB
        if img.ndim == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
        # Add to list
        images.append(img)
    
    # Try to extract exposure times from EXIF data
    # For simplicity, we'll use a doubling sequence if EXIF data is not available
    try:
        import exifread
        exposure_times = []
        for file_path in file_paths:
            with open(file_path, 'rb') as f:
                tags = exifread.process_file(f)
                if 'EXIF ExposureTime' in tags:
                    exp_time = tags['EXIF ExposureTime'].values[0]
                    # Convert fraction to float
                    if isinstance(exp_time, exifread.utils.Ratio):
                        exp_time = float(exp_time.num) / float(exp_time.den)
                    else:
                        exp_time = float(exp_time)
                    exposure_times.append(exp_time)
                else:
                    # Fall back to doubling sequence
                    raise ValueError("ExposureTime tag not found")
    except (ImportError, ValueError):
        # If exifread not installed or EXIF data not available, use doubling sequence
        print("Could not extract exposure times from EXIF data. Using doubling sequence...")
        exposure_times = [2**i for i in range(len(images))]
    
    print(f"Exposure times: {exposure_times}")
    
    return images, exposure_times


def preprocess_raw_images(raw_images):
    """
    Preprocess RAW images for HDR merging (normalization, etc.).
    
    Args:
        raw_images (list): List of RAW image arrays
        
    Returns:
        list: Preprocessed images ready for HDR merging
    """
    processed_images = []
    
    for img in raw_images:
        # Convert to float32
        img_float = img.astype(np.float32)
        
        # Normalize to [0, 1] range
        if img_float.max() > 0:
            img_float = img_float / img_float.max()
        
        processed_images.append(img_float)
    
    return processed_images

def preprocess_jpeg_images(jpeg_images, exposure_times):
    """
    Preprocess JPEG images for HDR merging (linearization, etc.).
    
    Args:
        jpeg_images (list): List of JPEG image arrays
        exposure_times (list): List of exposure times
        
    Returns:
        list: Preprocessed (linearized) images ready for HDR merging
    """
    # Normalize JPEG images to [0, 1]
    normalized_images = [img.astype(np.float32) / 255.0 for img in jpeg_images]
    
    # Recover camera response function
    calibration = ResponseCalibration(normalized_images, exposure_times)
    g = calibration.solve_g(n_samples=100)
    
    # Linearize images
    linearized_images = []
    for img in normalized_images:
        # Convert to uint8 for indexing into g
        img_uint8 = np.clip(img * 255, 0, 255).astype(np.uint8)
        linear_img = calibration.linearize_image(img_uint8, g)
        linearized_images.append(linear_img)
    
    return linearized_images, g

In [18]:
## Create two HDR images (one from RAW and one from JPEG imgs) using the above implemented algorithms
def create_hdr_image(images, exposure_times, weighting_method='gaussian', merging_method='logarithmic'):
    """
    Create an HDR image from an exposure stack.
    
    Args:
        images (list): List of preprocessed images
        exposure_times (list): List of exposure times
        weighting_method (str): Weighting method ('uniform', 'tent', 'gaussian', 'photon')
        merging_method (str): Merging method ('linear', 'logarithmic')
        
    Returns:
        numpy.ndarray: HDR image
    """
    # Create HDR merger
    merger = HDRMerger(weighting_method=weighting_method)
    
    # Merge images using specified method
    if merging_method == 'linear':
        hdr_image = merger.linear_merging(images, exposure_times)
    else:
        hdr_image = merger.logarithmic_merging(images, exposure_times)
    
    return hdr_image

def save_hdr_image(hdr_image, filename):
    """
    Save HDR image to file.
    
    Args:
        hdr_image (numpy.ndarray): HDR image
        filename (str): Output filename
    """
    # Convert RGB to BGR for OpenCV
    bgr_image = cv2.cvtColor(hdr_image, cv2.COLOR_RGB2BGR)
    
    # Save as HDR file
    cv2.imwrite(filename, bgr_image)
    print(f"Saved HDR image to {filename}")


In [19]:
## Tonemap the images with the above-implemented algorithms (in Part 3). Experiment with different parameters. Show some tonemaps

def apply_tone_mapping(hdr_image, method='luminance', key=0.18, burn=0.85, gamma=2.2):
    """
    Apply tone mapping to HDR image.
    
    Args:
        hdr_image (numpy.ndarray): HDR image
        method (str): Tone mapping method ('rgb' or 'luminance')
        key (float): Key value (brightness)
        burn (float): Burn value (highlight compression)
        gamma (float): Gamma value for gamma correction
        
    Returns:
        numpy.ndarray: Tone mapped LDR image
    """
    # Create tone mapper with specified parameters
    tone_mapper = ToneMapper(key=key, burn=burn)
    
    # Apply tone mapping
    ldr_image = tone_mapper.tone_map(hdr_image, method=method, apply_gamma_correction=True)
    
    return ldr_image

def experiment_with_tone_mapping(hdr_image, output_dir='tone_mapping_results'):
    """
    Experiment with different tone mapping parameters and save results.
    
    Args:
        hdr_image (numpy.ndarray): HDR image
        output_dir (str): Directory to save results
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create tone mapper
    tone_mapper = ToneMapper()
    
    # Parameters to experiment with
    methods = ['luminance', 'rgb']
    key_values = [0.09, 0.18, 0.36]
    burn_values = [0.7, 0.85, 1.0]
    
    # Create a figure to display results
    fig = plt.figure(figsize=(15, 10))
    
    idx = 1
    for method in methods:
        for key in key_values:
            for burn in burn_values:
                # Set parameters
                tone_mapper.key = key
                tone_mapper.burn = burn
                
                # Apply tone mapping
                ldr_image = tone_mapper.tone_map(hdr_image, method=method)
                
                # Save image
                filename = f"{output_dir}/tonemap_{method}_key{key}_burn{burn}.png"
                tone_mapper.save_tone_mapped_image(ldr_image, filename)
                
                # Add to figure
                plt.subplot(len(methods), len(key_values) * len(burn_values), idx)
                plt.imshow(ldr_image)
                plt.title(f"{method}, key={key}, burn={burn}")
                plt.axis('off')
                idx += 1
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/tone_mapping_comparison.png", dpi=300)
    plt.show()



ef find_best_tone_mapping_params(hdr_image, method='luminance'):
    """
    Interactive tool to find the best tone mapping parameters for an HDR image.
    
    Args:
        hdr_image (numpy.ndarray): HDR image
        method (str): Tone mapping method ('rgb' or 'luminance')
        
    Returns:
        tuple: Best parameters (key, burn, gamma)
    """
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Create tone mapper
    tone_mapper = ToneMapper()
    
    # Initial parameters
    key = 0.18
    burn = 0.85
    gamma = 2.2
    
    # Apply tone mapping with initial parameters
    ldr_image = tone_mapper.tone_map(hdr_image, method=method)
    
    # Display image
    plt.imshow(ldr_image)
    plt.title(f"Method: {method}, Key: {key:.2f}, Burn: {burn:.2f}, Gamma: {gamma:.2f}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Get user input for parameters
    print("\nEnter parameters (press Enter to keep current value):")
    
    try:
        key_input = input(f"Key (current={key:.2f}): ")
        if key_input:
            key = float(key_input)
        
        burn_input = input(f"Burn (current={burn:.2f}): ")
        if burn_input:
            burn = float(burn_input)
        
        gamma_input = input(f"Gamma (current={gamma:.2f}): ")
        if gamma_input:
            gamma = float(gamma_input)
    except ValueError:
        print("Invalid input, using current values.")
    
    # Update tone mapper with new parameters
    tone_mapper.key = key
    tone_mapper.burn = burn
    
    # Apply tone mapping with new parameters
    ldr_image = tone_mapper.tone_map(hdr_image, method=method)
    
    # Display final image
    plt.figure(figsize=(12, 8))
    plt.imshow(ldr_image)
    plt.title(f"Method: {method}, Key: {key:.2f}, Burn: {burn:.2f}, Gamma: {gamma:.2f}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"Final parameters: Key={key:.2f}, Burn={burn:.2f}, Gamma={gamma:.2f}")
    
    return key, burn, gamma

SyntaxError: invalid syntax (801502394.py, line 75)

In [ ]:
def compare_raw_jpeg_hdr(raw_hdr, jpeg_hdr, output_file='hdr_comparison.png'):
    """
    Compare HDR images created from RAW and JPEG.
    
    Args:
        raw_hdr (numpy.ndarray): HDR image from RAW
        jpeg_hdr (numpy.ndarray): HDR image from JPEG
        output_file (str): Output filename for comparison image
    """
    # Create tone mapper for display
    tone_mapper = ToneMapper()
    
    # Tone map both images with the same parameters
    raw_ldr = tone_mapper.tone_map(raw_hdr)
    jpeg_ldr = tone_mapper.tone_map(jpeg_hdr)
    
    # Create figure for comparison
    plt.figure(figsize=(15, 7))
    
    plt.subplot(1, 2, 1)
    plt.imshow(raw_ldr)
    plt.title('HDR from RAW')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(jpeg_ldr)
    plt.title('HDR from JPEG')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()

In [ ]:
def bonus_workflow_example():
    """
    Example workflow for the bonus task.
    """
    # 1. Load exposure stack
    print("Step 1: Load exposure stacks")
    raw_images, raw_exposures = load_exposure_stack('your_raw_directory', 'RAW')
    jpeg_images, jpeg_exposures = load_exposure_stack('your_jpeg_directory', 'JPEG')
    
    # 2. Preprocess images
    print("\nStep 2: Preprocess images")
    processed_raw = preprocess_raw_images(raw_images)
    processed_jpeg, response_curve = preprocess_jpeg_images(jpeg_images, jpeg_exposures)
    
    # 3. Create HDR images
    print("\nStep 3: Create HDR images")
    raw_hdr = create_hdr_image(processed_raw, raw_exposures, 'gaussian', 'logarithmic')
    jpeg_hdr = create_hdr_image(processed_jpeg, jpeg_exposures, 'gaussian', 'logarithmic')
    
    # 4. Save HDR images
    print("\nStep 4: Save HDR images")
    save_hdr_image(raw_hdr, 'raw_hdr.hdr')
    save_hdr_image(jpeg_hdr, 'jpeg_hdr.hdr')
    
    # 5. Compare HDR images
    print("\nStep 5: Compare HDR images")
    compare_raw_jpeg_hdr(raw_hdr, jpeg_hdr)
    
    # 6. Experiment with tone mapping
    print("\nStep 6: Experiment with tone mapping")
    experiment_with_tone_mapping(raw_hdr, 'raw_tone_mapping')
    experiment_with_tone_mapping(jpeg_hdr, 'jpeg_tone_mapping')
    
    # 7. Find best tone mapping parameters
    print("\nStep 7: Find best tone mapping parameters for RAW HDR")
    raw_key, raw_burn, raw_gamma = find_best_tone_mapping_params(raw_hdr)
    
    print("\nStep 8: Find best tone mapping parameters for JPEG HDR")
    jpeg_key, jpeg_burn, jpeg_gamma = find_best_tone_mapping_params(jpeg_hdr)
    
    # 9. Create final tone-mapped images with best parameters
    print("\nStep 9: Create final tone-mapped images")
    final_raw_ldr = apply_tone_mapping(raw_hdr, 'luminance', raw_key, raw_burn, raw_gamma)
    final_jpeg_ldr = apply_tone_mapping(jpeg_hdr, 'luminance', jpeg_key, jpeg_burn, jpeg_gamma)
    
    # 10. Save final images
    print("\nStep 10: Save final images")
    cv2.imwrite('final_raw_tonemap.png', cv2.cvtColor((final_raw_ldr * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))
    cv2.imwrite('final_jpeg_tonemap.png', cv2.cvtColor((final_jpeg_ldr * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))
    
    print("\nWorkflow complete!") 

##### *Discuss the above-generated images, commenting on which one you like the best.*

### **REFERENCES**
[1] P. E. Debevec and J. Malik. Recovering high dynamic range radiance maps from photographs.
In Proceedings of the 24th Annual Conference on Computer Graphics and Interactive Techniques,
SIGGRAPH ’97, pages 369–378, New York, NY, USA, 1997. ACM Press/Addison-Wesley Publishing
Co.

[2] E. Reinhard, M. Stark, P. Shirley, and J. Ferwerda. Photographic tone reproduction for digital
images. In Proceedings of the 29th Annual Conference on Computer Graphics and Interactive
Techniques, SIGGRAPH ’02, pages 267–276, New York, NY, USA, 2002. ACM.
